[PHE criteria for shielded patients](https://www.gov.uk/government/publications/covid-19-guidance-on-social-distancing-and-for-vulnerable-people/guidance-on-social-distancing-for-everyone-in-the-uk-and-protecting-older-people-and-vulnerable-adults) specifies people with weakened immune systems and goes on to give an example of "_steroid tablets_". This is very unspecific as you can have injections, enemeas, inhaled which can all act on the imune system. The [BNF has a useful summary](https://bnf.nice.org.uk/treatment-summary/corticosteroids-general-use.html) of steroids including area of action, differnt types, uses and side-effects. In this list we will include "replacement like" steroids. 


In [1]:
from ebmdatalab import bq
import os
import pandas as pd

In [2]:
sql = '''WITH bnf_codes AS (
  SELECT bnf_code FROM hscic.presentation WHERE 
    bnf_code LIKE '060302%'       OR    #BNF  glucocorticoid
    bnf_code LIKE '060301%'       OR    #BNF replacement therapy - fludrocortisone
    bnf_code LIKE '100102%'       OR      #BNF paragraph on corticosteroinds in rheumatic disease
    bnf_code LIKE '010502%'             #BNF paragraph on corticosteroinds in bowel disorders
  )

SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp_full
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, bnf_code, id'''

replacement_steroids_codelist = bq.cached_read(sql, csv_path=os.path.join('..','data','replacement_steroids_codelist.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
replacement_steroids_codelist.count()

type        524
id          524
bnf_code    524
nm          524
dtype: int64

In [3]:
sql = '''
SELECT obj_type, snomed_id, bnf_code, dmd_name, form_route
FROM ebmdatalab.measures_v2.dmd_objs_with_form_route
WHERE 
    (bnf_code LIKE '060302%'       OR    #BNF  glucocorticoid
    bnf_code LIKE '060301%'       OR    #BNF replacement therapy - fludrocortisone
    bnf_code LIKE '100102%'       OR      #BNF paragraph on corticosteroinds in rheumatic disease
    bnf_code LIKE '010502%' )            #BNF paragraph on corticosteroinds in bowel disorders
AND
(obj_type = "vmp" OR obj_type = "amp")


ORDER BY obj_type, bnf_code'''

test2 = bq.cached_read(sql, csv_path=os.path.join('..','data','test2.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
test2.count()

obj_type      804
snomed_id     804
bnf_code      804
dmd_name      804
form_route    804
dtype: int64

In [4]:
sql = '''WITH subquery AS (
  SELECT
    obj_type,
    snomed_id,
    bnf_code,
    dmd_name,
    form_route
  FROM measures.dmd_objs_with_form_route
  WHERE (
      bnf_code LIKE '060302%' OR # BNF glucocorticoid
      bnf_code LIKE '060301%' OR # BNF replacement therapy - fludrocortisone
      bnf_code LIKE '100102%' OR # BNF paragraph on corticosteroinds in rheumatic disease
      bnf_code LIKE '010502%'    # BNF paragraph on corticosteroinds in bowel disorders
    ) AND (
      obj_type = "vmp" OR
      obj_type = "amp"
    )
)

SELECT *
FROM subquery
WHERE snomed_id IN (
  SELECT snomed_id
  FROM subquery
  GROUP BY snomed_id
  HAVING COUNT(snomed_id) > 1
)

ORDER BY obj_type, dmd_name, form_route'''

all_steroids = bq.cached_read(sql, csv_path=os.path.join('..','data','all_steroids.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
all_steroids.count()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=7JlaG4J9PsHHllytWtZg1whrBsmFfh&prompt=consent&access_type=offline


Enter the authorization code:  4/ygG6uehdCYqrFNd7UnqMZb31TF7u0kQxLp8NUne6aIDIX6RcKl_rosY


Downloading: 100%|██████████| 374/374 [00:00<00:00, 1849.09rows/s]


obj_type      374
snomed_id     374
bnf_code      374
dmd_name      374
form_route    374
dtype: int64